In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import load_model
classifier = load_model('thaiseg_300krows_8timestep_bidirection.h5')


Using TensorFlow backend.


In [7]:
# Importing the training set
df = pd.read_csv('timeStep8CorpusFull.csv',encoding='utf-8')
print(df.shape)
df = df.head(300000)
df.head(10)

(1172847, 10)


,Unnamed: 0,0,1,2,3,4,5,6,7,isEnd
0,0,ม,ี,ค,ำ,ถ,า,ม,ม,0
1,1,ี,ค,ำ,ถ,า,ม,ม,า,0
2,2,ค,ำ,ถ,า,ม,ม,า,ก,0
3,3,ำ,ถ,า,ม,ม,า,ก,ม,0
4,4,ถ,า,ม,ม,า,ก,ม,า,0
5,5,า,ม,ม,า,ก,ม,า,ย,1
6,6,ม,ม,า,ก,ม,า,ย,พ,0
7,7,ม,า,ก,ม,า,ย,พ,ร,0
8,8,า,ก,ม,า,ย,พ,ร,ั,0
9,9,ก,ม,า,ย,พ,ร,ั,่,0


# X and y
X = df[['0','1','2','3','4','5','6','7']]
y = df[['isEnd']]

#Label each character
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X[['0']])
X = X.apply(le.transform)
maxChar = 65

In [11]:
timeStep=8
def thaiSegment(paragraph):
    sentences = paragraph.split(" ")
    all_words = []
    for sentence in sentences:
        test = sentence[-timeStep+1:]+sentence #Same trick as in CNN
        df = {}
        for i in range(timeStep):
            if(i-timeStep+1<0):
                df[i] = list(test[i:i-timeStep+1])
            else:
                df[i] = list(test[i:])
        df_test = pd.DataFrame(df)
        df_test = df_test.apply(le.transform).values
        df_test = (np.arange(maxChar) == df_test[:,:,None]-1).astype(int)
        y_pred = classifier.predict(df_test)
        y_pred = y_pred>0.5

        df_test  = pd.DataFrame(df)
        df_test['isEnd'] = y_pred

        words = []
        w = ''

        for i in range(len(test)-timeStep+1):
            c = test[i+timeStep-1]
            w+= c
            if(y_pred[i][0]==True):
                words.append(w)
                w=''

        words.append(w)
        if "" in words: 
            words.remove('')
        all_words.append("|".join(words))
    return " ".join(all_words)


In [1]:
def compareToken2Real(tokenized,real):
    test = tokenized
    correct = 0
    incorrect = 0
    i=0
    j=0
    if(len(tokenized)<200): print(tokenized)
    cm = [[0,0],[0,0]]
    print("Start comparing")
    while(i<len(real) and j<len(test)):
        if(real[i]=="|" and test[j]=="|"):#True Positive
            cm[0][0]+=1
            i+=1
            j+=1
        elif(real[i]==test[j]):#True Negative
            cm[1][1]+=1
            i+=1
            j+=1
        elif(real[i]=="|"):#False Negative
            cm[0][1]+=1
            i+=2
            j+=1
        elif(test[j]=="|"):#False Positive
            cm[1][0]+=1
            j+=2
            i+=1
    n = cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0]
    acc = (cm[0][0]+cm[1][1])/n
    precision = cm[0][0]/(cm[0][0]+cm[1][0])
    recall = cm[0][0]/(cm[0][0]+cm[0][1])
    n = cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0]
    acc = (cm[0][0]+cm[1][1])/n
    precision = cm[0][0]/(cm[0][0]+cm[1][0])
    recall = cm[0][0]/(cm[0][0]+cm[0][1])
    
    print("Accuracy : {:0.2f}".format(acc*100))
    print("Precision : {:0.2f}".format(precision*100))
    print("Recall : {:0.2f}".format(recall*100))
    
    if(recall+precision!=0):
        f1 =((2*precision*recall)/(precision+recall))*100
        print("F1 Score : {:0.2f}".format(f1))
    else:
        f1 = 0
        print("F1 Score : 0.00")
    return (acc,precision,recall,f1)

In [12]:
def compareTokenizerScore(sentence='สวัสดี|ครับ|ผม|ชื่อ|เคอิโงะ'):
    raw_sentence = sentence.replace("|","")

    tokenized4 = thaiSegment(raw_sentence)


    compareToken2Real(tokenized4,sentence)
    
    
compareTokenizerScore('ปกปิด|ข้อมูล|การ|ระบาด|ของ|โรค|ต่อ|เกษตรกร|จน|ทำ|ให้|ไก่|ที่|เกษตรกร|เลี้ยง|ไว้|เป็น|ราย|ได้|หลัก|ของ|ครัวเรือน|ต้อง|ตาย|เป็น|จำนวน|มหาศาล')

ปกปิด|ข้อมูล|การ|ระบาด|ของ|โรคต่อเกษตรกรจน|ทำ|ให้|ไก่|ที่|เกษตรกรเลี้ยง|ไว้|เป็น|ราย|ได้|หลัก|ของ|ครัว|เรือน|ต้องตาย|เป็น|จำนวน|มหาศาล
Start comparing
Accuracy : 95.49
Precision : 95.45
Recall : 80.77
F1 Score : 87.50


In [14]:
#Compare with other tokenizer
def convertLetter(row):
    return row['7'] if row['isEnd']==0 else row['7']+"|"
df['letter'] = df.apply(convertLetter,axis=1)
testString = "".join(df['letter'].values)

In [25]:
print("Test set 1")
compareTokenizerScore(testString[:100000])
print("Test set 2")
compareTokenizerScore(testString[100001:220000])
print("Test set 3")
compareTokenizerScore(testString[250001:300000])

Test set 1
Start comparing
Accuracy : 96.76
Precision : 90.18
Recall : 94.02
F1 Score : 92.06
Test set 2
Start comparing
Accuracy : 96.51
Precision : 90.35
Recall : 92.61
F1 Score : 91.47
Test set 3
Start comparing
Accuracy : 96.50
Precision : 90.79
Recall : 92.27
F1 Score : 91.52


In [15]:
print("Test set 4")
compareTokenizerScore(testString[320001:420000])

Test set 4
Start comparing
Accuracy : 96.08
Precision : 89.24
Recall : 91.87
F1 Score : 90.53


In [28]:
compareTokenizerScore("พรุ่งนี้|จะ|ไป|โรงเรียน|แล้ว อย่า|ตื่น|สาย|นะ")

พรุ่ง|นี้|จะ|ไป|โรงเรียน|แล้ว อย่าตื่น|สาย|นะ
Start comparing
Accuracy : 95.45
Precision : 85.71
Recall : 85.71
F1 Score : 85.71


In [30]:
compareTokenizerScore("ถ้า|เทอ|ไม่|ปัย|กิน|ข้าว|กับ|ชั้น ชั้น|จะ|ฟ้อง|ป๊า")

ถ้า|เทอไม่|ปัย|กิน|ข้าว|กับ|ชั้น ชั้น|จะ|ฟ้อง|ป๊า
Start comparing
Accuracy : 97.96
Precision : 100.00
Recall : 90.00
F1 Score : 94.74


In [31]:
compareTokenizerScore("คัย|หน้า|นัย|ก้อ|หยุด|เรา|มัย|ดั้ย|หร้อก")

คัย|หน้า|นัย|ก้อหยุด|เรา|มัย|ดั้ย|หร้อก
Start comparing
Accuracy : 97.44
Precision : 100.00
Recall : 87.50
F1 Score : 93.33


In [36]:
compareTokenizerScore("สนุก|มากกกกกกกก|เลยยยยยยยยย")

สนุกมา|ก|กกกกกกกเลย|ยย|ยย|ยยยย
Start comparing
Accuracy : 72.00
Precision : 0.00
Recall : 0.00
F1 Score : 0.00


In [42]:
compareTokenizerScore("ครู|สอน|คณิตศาสตร์|สอน|ยากจุง|เบย")

ครูสอน|คณิตศาสตร์|สอน|ยากจุง|เบย
Start comparing
Accuracy : 96.88
Precision : 100.00
Recall : 80.00
F1 Score : 88.89
